In [1]:
import os
import pandas as pd
import numpy as np
import openpyxl
import xlsxwriter

from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.chart.data import CategoryChartData, ChartData, XyChartData
from pptx.enum.chart import XL_CHART_TYPE, XL_LABEL_POSITION, XL_TICK_MARK
from pptx.dml.color import RGBColor
from pptx.enum.text import PP_ALIGN
from pandas import DataFrame

In [2]:
#import calc template or raw data, create list of reach data and pass through variable  

# Input Required: Select Data Input File and PPT Template

In [3]:
data_location = (r"C:\Users\resissma2001\Documents\CI\ShareShift\Facebook\2021\Quickbooks\Quickbooks Facebook Shareshift Calculator.xlsx")
prs = Presentation(r"C:\Users\resissma2001\Documents\CI\Automation PPT Tools\Share Shift\Google Shareshift - Template (1).pptx")

# Functions

In [4]:
def read_files(sheet_name):
    
    '''Read in excel files for specific tab
    
    Args: name of sheet in excel
    
    Returns: dataframe
    
    '''
    
    df = pd.read_excel(data_location,sheet_name).reset_index(drop=True)
    return df

In [5]:
def replace_text(search_str, repl_str):
    """Takes dict of {search_str: repl_str, ... } and replaces all matches.
    Currently not implemented for charts or graphics.
    Search and replace text in PowerPoint while preserving formatting"""
    for slide in prs.slides:
        for shape in slide.shapes:
            if shape.has_text_frame:
                if (shape.text.find(search_str)) != -1:
                    text_frame = shape.text_frame
                    for paragraph in text_frame.paragraphs:
                        for run in paragraph.runs:
                            cur_text = run.text
                            new_text = cur_text.replace(str(search_str), str(repl_str))
                            run.text = new_text
            if shape.has_table:
                for row in shape.table.rows:
                    for cell in row.cells:
                        if search_str in cell.text:
                            new_text = cell.text.replace(str(search_str), str(repl_str))
                            cell.text = new_text
                            cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

In [6]:
def create_footnote(slide_num):
    footnote = slide_num.placeholders[15]
    footnote.text = campaign_sourcing
    return footnote

# Input Required: Define Campaign Information (Segments, Target, Name, Dates)

In [7]:
#Target demo: Write it out as you would like it to appear in the deck, e.g. P18-99 becomes P18+. 
#Fusion Period: Use the reference period from NMI and put in the month and the year
#TV Data Source: Nielsen Ad Intel or Google
target_demo = 'Persons 18-34'
campaign_name = 'Quickbooks'
current_month = 'August 2021'
campaign_dates = '1/1/2021 - 4/20/2021'
advertiser= 'Quickbooks'
frequency_cap= 'No Capping'
fusion_period='March 2021'
tv_data_source='Google'

In [8]:
if tv_data_source=='Google':
    data_source='provided by Google'
else:
    data_source='measured by Ad Intel'

In [9]:
prepared_for= "Prepared for "+ advertiser

# Read in Data

In [10]:
budget_allocation = read_files('Budget Allocations (Slide 2)')
exec_summary = read_files('Exec Summary (Slide 3)')
full_reach = pd.read_excel(data_location,'Full Reach (Slide 4)', skiprows=2)
scenarios = read_files('Scenario Amounts (under charts)')
reach_data = pd.read_excel(data_location,'Reach (Slide 5)', skiprows=3)
frequency = pd.read_excel(data_location,'Freq (Slide 6)', skiprows=3) 
cost_per_reach = pd.read_excel(data_location,'Cost per Reach (Slide 7)', skiprows=3)
cross_platform = read_files ('Cross Platform (Slide 8)')
cost_per_reach_thousands= read_files ('Cost per Reach (000) (Slide 9)')
summary_table_data = read_files('Summary Table (Slide 10)')
scenario_amounts= pd.read_excel(data_location, 'Scenario Amounts (under charts)', skiprows=10)

# Formatting Data Files

In [11]:
#formatting cost efficency data
cost_per_reach_thousands= cost_per_reach_thousands.iloc[[4,5,6,7],:]
print(cost_per_reach_thousands)

             Unnamed: 0   Unnamed: 1   Unnamed: 2   Unnamed: 3  Unnamed: 4  \
4  Sum of Original Plan  4.60682e+07  4.05366e+07  5.53161e+06         NaN   
5         Sum of Plan 1  4.60682e+07  3.37866e+07  1.22816e+07         NaN   
6         Sum of Plan 2  4.60682e+07  2.70366e+07  1.90316e+07         NaN   
7         Sum of Plan 3  4.60682e+07  2.02866e+07  2.57816e+07         NaN   

  Unnamed: 5 Unnamed: 6 Unnamed: 7  
4    806.705    1051.55    139.724  
5    758.641    896.185    253.162  
6    748.516    738.591    372.676  
7    745.169    577.889    490.649  


In [12]:
#Formatting the cross platform data
cross_platform= cross_platform.iloc[[5,6,7,8],:]
cross_platform

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
5,Sum of Original Plan,24.26,25.72,29.13,NaN,24.26,25.72,29.13
6,Sum of Plan 1,16.91,31.89,35.3,NaN,16.91,31.89,35.3
7,Sum of Plan 2,14.51,34.54,36.18,NaN,14.51,34.54,36.18
8,Sum of Plan 3,12.85,37,35.77,NaN,12.85,37,35.77


In [13]:
print(full_reach)

       Unnamed: 0        Dollars   Dollars (MM)  Reach %
0   Original Plan  Original Plan  Original Plan   0.7909
1         Shift 1      4,053,663        4.05366   0.8323
2         Shift 2      8,107,326        8.10733   0.8442
3         Shift 3     12,160,989         12.161   0.8508
4         Shift 4     16,214,652        16.2147   0.8545
5         Shift 5     20,268,316        20.2683   0.8562
6         Shift 6     24,321,979         24.322   0.8559
7         Shift 7     28,375,642        28.3756   0.8536
8         Shift 8     32,429,305        32.4293   0.8482
9         Shift 9     36,482,968         36.483   0.8355
10       Shift 10       40536631        40.5366   0.7592


In [14]:
#formatting reach data
reach_data['Unnamed: 1']=reach_data['Unnamed: 1'].div(100)
#Keep necessary rows in cpp table
cost_per_reach= cost_per_reach.iloc[[0,1,2,3],:]

In [15]:
summary_table_clean=summary_table_data.iloc[46:71,0:5]
summary_table_clean = summary_table_clean.dropna(subset = ["Metric"]).reset_index(drop = True)

In [16]:
campaign_sourcing = 'Source: Nielsen Media Impact, Nielsen Ad Intel, Nielsen TV/Digital Fusion data'+', '+campaign_name+', '+campaign_dates

# Search and Replace Variables

In [17]:
# % of total budget to YT
original_total_budget= str(round(budget_allocation.iat[11,2]*100))+'%'
scenarioone_total_budget=str(round(budget_allocation.iat[12,3]*100))+'%'
scenariotwo_total_budget=str(round(budget_allocation.iat[13,3]*100))+'%'
scenariothree_total_budget=str(round(budget_allocation.iat[14,3]*100))+'%'

In [18]:
budget_allocation

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,Metric,Budget,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,Column Labels,NaN,NaN
3,Values,Total Plan,TV Programs,Digital
4,Sum of Original Plan,4.60682e+07,4.05366e+07,5.53161e+06
5,Sum of Plan 1,4.60682e+07,3.37866e+07,1.22816e+07
6,Sum of Plan 2,4.60682e+07,2.70366e+07,1.90316e+07
7,Sum of Plan 3,4.60682e+07,2.02866e+07,2.57816e+07
8,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN


In [19]:
# YT % of total budget
scenarioone_yt_budget=str(round(budget_allocation.iat[12,2]*100))+'%'
scenariotwo_yt_budget=str(round(budget_allocation.iat[13,2]*100))+'%'
scenariothree_yt_budget=str(round(budget_allocation.iat[14,2]*100))+'%'

In [20]:
#exec summary 
exec_summary_callout_one=str(int(round(exec_summary.iat[0,2]*100)))+'%'
exec_summary_callout_three=str(int(round(exec_summary.iat[4,2]*-100)))+'%'

In [21]:
exec_summary_callout_two= int(round(exec_summary.iloc[2,2]*100))

if exec_summary_callout_two>0:
    exec_summary_callout_two =str(exec_summary_callout_two)+'% increase'
else:
     exec_summary_callout_two=str(exec_summary_callout_two*-1) +'% decrease'

In [22]:
#full reach

full_reach_callout_clean=full_reach.sort_values(by=['Reach %'], ascending=False)
full_reach_callout_clean=full_reach_callout_clean.reset_index(drop=True)

full_reach_callout='$'+str(round(full_reach_callout_clean.iat[0,2],1))+'M'

full_reach_callout

'$20.3M'

In [23]:
#scenarios

scenarios_total='$'+ str(round(scenarios.iat[10,2]/1000000,1))+'M'
original_tv= '$'+ str(round(scenarios.iat[11,2]/1000000,1))+'M'
original_yt= '$'+ str(round(scenarios.iat[12,2]/1000000,1))+'M'

scenario_one_tv='$'+ str(round(scenarios.iat[11,3]/1000000,1))+'M'
scenario_one_yt='$'+ str(round(scenarios.iat[12,3]/1000000,1))+'M'

scenario_two_tv='$'+ str(round(scenarios.iat[11,4]/1000000,1))+'M'
scenario_two_yt='$'+ str(round(scenarios.iat[12,4]/1000000,1))+'M'

scenario_three_tv='$'+ str(round(scenarios.iat[11,5]/1000000,1))+'M'
scenario_three_yt='$'+ str(round(scenarios.iat[12,5]/1000000,1))+'M'

In [24]:
summary_scenario_max_shift='$'+str(round(summary_table_data.iat[3,1]/1000000-summary_table_data.iat[3,4]/1000000,1))+'M'

In [25]:
#reachdata
reach_data_callout=str(round(reach_data.iat[3,4]*100,1))+'%'

In [26]:
#frequency
frequency_callout_increaseordecrease= frequency.iloc[3,5]

if frequency_callout_increaseordecrease>0:
    frequency_callout_increaseordecrease= 'increase'
else:
    frequency_callout_increaseordecrease= 'decrease'
frequency_callout_increaseordecrease


frequency_callout= frequency.iloc[3,5]

if frequency_callout<0:
    frequency_callout=str(round(frequency_callout*-100,1))+'%'
else:
     frequency_callout=str(round(frequency_callout*100,1))+'%'

In [27]:
#cost per reach
cost_per_reach_callout=str(round(cost_per_reach.iat[3,7]*-100,1))+'%'

In [28]:
summary_scenario_one_shift= 'Scenario #1                       ' +'$'+str(round(summary_table_data.iat[3,1]/1000000-summary_table_data.iat[3,2]/1000000,1))+'M Shifted to YT'
summary_scenario_two_shift= 'Scenario #2                       ' +'$'+str(round(summary_table_data.iat[3,1]/1000000-summary_table_data.iat[3,3]/1000000,1))+'M Shifted to YT'
summary_scenario_three_shift='Scenario #3                       ' + '$'+str(round(summary_table_data.iat[3,1]/1000000-summary_table_data.iat[3,4]/1000000,1))+'M Shifted to YT'


# Slide 1: Title 

In [29]:
title_slide = prs.slides[0]
subtitle1 = title_slide.placeholders[1]
subtitle1.text = campaign_name

date_footer = title_slide.placeholders[17]
date_footer.text = "Prepared for Google" + current_month

# Slide 4: Full Reach

In [30]:
full_reach

,Unnamed: 0,Dollars,Dollars (MM),Reach %
0,Original Plan,Original Plan,Original Plan,0.7909
1,Shift 1,"4,053,663",4.05366,0.8323
2,Shift 2,"8,107,326",8.10733,0.8442
3,Shift 3,"12,160,989",12.161,0.8508
4,Shift 4,"16,214,652",16.2147,0.8545
5,Shift 5,"20,268,316",20.2683,0.8562
6,Shift 6,"24,321,979",24.322,0.8559
7,Shift 7,"28,375,642",28.3756,0.8536
8,Shift 8,"32,429,305",32.4293,0.8482
9,Shift 9,"36,482,968",36.483,0.8355


In [31]:
fullreachslide=prs.slides[3]
fullreachchart=fullreachslide.shapes[3].chart


In [32]:
fullreach_list=[]
fullreach_list= full_reach['Reach %'].to_list()

In [33]:
fullreachcategories_list=[]
fullreachcategories_list= full_reach['Dollars (MM)'].to_list()

In [34]:
for number in range(1,11): 
    fullreachcategories_list[number]=str(round(fullreachcategories_list[number],1))+'M'

In [35]:
fullreachchartdata=ChartData()
fullreachchartdata.categories=fullreachcategories_list
fullreachchartdata.add_series("Reach", (fullreach_list))
fullreachchart.replace_data(fullreachchartdata)

In [36]:
fullreachchartdata

In [37]:
create_footnote(fullreachslide)

# Slide 5: Reach 

In [38]:
#selects the slide number (starts counting from 0)
reachslide=prs.slides[4]
#gets the shape number index start backwards on selection pane (chart tools-->format-->selection pane)
reachchart=reachslide.shapes[3].chart

#create list and fill list with Unnamed column in the reach chart
reach_list=[]
reach_list= reach_data['Unnamed: 1'].to_list()



#empty chart variable
reachchartdata=ChartData()
reachchartdata.categories=["Original Plan", "Scenario 1", "Scenario 2", "Scenario 3"]
reachchartdata.add_series('Reach %', (reach_list))



reachchart.replace_data(reachchartdata)

In [39]:
reach_data

,Values,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,Sum of Original Plan,0.7909,NaN,NaN,NaN
1,Sum of Plan 1,0.8410,NaN,NaN,NaN
2,Sum of Plan 2,0.8523,NaN,NaN,NaN
3,Sum of Plan 3,0.8562,NaN,% Increase in Reach,0.082564


In [40]:
reachchartdata.categories

In [41]:
create_footnote(reachslide)

# Slide 6: Frequency 

In [42]:
frequencyslide=prs.slides[5]
frequencychart=frequencyslide.shapes[4].chart


frequency_list=[]
frequency_list= frequency['Unnamed: 1'].to_list()


frequencychartdata=ChartData()
frequencychartdata.categories=["Original Plan", "Scenario 1", "Scenario 2", "Scenario 3"]
frequencychartdata.add_series("Frequency", (frequency_list))
frequencychart.replace_data(frequencychartdata)

In [43]:
create_footnote(frequencyslide)

# Slide 7: CPP

In [44]:
cppslide=prs.slides[6]
cppchart=cppslide.shapes[0].chart


cpp_list=[]
cpp_list= cost_per_reach['Cost per Reach'].to_list()


In [45]:
cpp_list

[582.478726775825, 547.7793400796671, 540.5167488055848, 538.0546893330996]

In [46]:
cppchartdata=ChartData()
cppchartdata.categories=["Original Plan", "Scenario 1", "Scenario 2", "Scenario 3"]
cppchartdata.add_series("CPP", (cpp_list))
cppchart.replace_data(cppchartdata)

In [47]:
create_footnote(cppslide)

# Slide 8: Cross-Platform

In [48]:
crossplatformslide= prs.slides[7]
crossplatformchart= crossplatformslide.shapes[4].chart


crossplatformtvonly_list=[]
crossplatformyoutubeonly_list=[]
crossplatformboth_list=[]

crossplatformtvonly_list= cross_platform['Unnamed: 5'].to_list()
crossplatformyoutubeonly_list= cross_platform['Unnamed: 6'].to_list()
crossplatformboth_list= cross_platform['Unnamed: 7'].to_list()

In [49]:
crossplatformchartdata=ChartData()

crossplatformchartdata.categories=["Original Plan", "Scenario 1", "Scenario 2", "Scenario 3"]
crossplatformchartdata.add_series('TV Only', (crossplatformtvonly_list))
crossplatformchartdata.add_series('TV and Youtube Overlap', (crossplatformboth_list))
crossplatformchartdata.add_series('Youtube Only', (crossplatformyoutubeonly_list))

crossplatformchart.replace_data(crossplatformchartdata)

In [50]:
create_footnote(crossplatformslide)

# Slide 9: CE by Scenarios

In [51]:
costefficiencyslide= prs.slides[8]
costefficiencychart= costefficiencyslide.shapes[4].chart


costefficiencytotal_list=[]
costefficiencytv_list=[]
costefficiencyyoutube_list=[]

costefficiencytotal_list= cost_per_reach_thousands['Unnamed: 5'].to_list()
costefficiencytv_list= cost_per_reach_thousands['Unnamed: 6'].to_list()
costefficiencyyoutube_list= cost_per_reach_thousands['Unnamed: 7'].to_list()


In [52]:
costefficiencychartdata=ChartData()
costefficiencychartdata.categories=["Original Plan", "Scenario 1", "Scenario 2", "Scenario 3"]
costefficiencychartdata.add_series('Total Campaign', (costefficiencytotal_list))
costefficiencychartdata.add_series('Total TV',    (costefficiencytv_list))
costefficiencychartdata.add_series('Total Youtube', (costefficiencyyoutube_list))

costefficiencychart.replace_data(costefficiencychartdata)

In [53]:
create_footnote(costefficiencyslide)

# Slide 10: Summary Slide

In [54]:
summaryslide= prs.slides[9]

In [55]:
summarytable = summaryslide.shapes[8].table

for col in range(1, len(summarytable.columns)):
    for row in range(0, len(summarytable.rows)):
        if row in [1,7,16]:
            summarytable.cell(row, col).text_frame.text = '$'+str(round((summary_table_clean.iloc[row-1, col]/1000000),1))+'M'
            summarytable.cell(row, col).text_frame.paragraphs[0].font.size = Pt(8)
            summarytable.cell(row,col).text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
        elif row in [5,14,23]:
            summarytable.cell(row, col).text_frame.text = '$'+str(round((summary_table_clean.iloc[row-1, col]),2)) 
            summarytable.cell(row, col).text_frame.paragraphs[0].font.size = Pt(8)
            summarytable.cell(row,col).text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
        elif row in [2,9,18]:
            summarytable.cell(row, col).text_frame.text = str(round((summary_table_clean.iloc[row-1, col])))
            summarytable.cell(row, col).text_frame.paragraphs[0].font.size = Pt(8)
            summarytable.cell(row,col).text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
        elif row in [4,13,22]:
            summarytable.cell(row, col).text_frame.text = str(round((summary_table_clean.iloc[row-1, col]),1))
            summarytable.cell(row, col).text_frame.paragraphs[0].font.size = Pt(8)
            summarytable.cell(row,col).text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
        elif row in [8,17]:
            summarytable.cell(row, col).text_frame.text = str(round((summary_table_clean.iloc[row-1, col])*100,1))+'%'
            summarytable.cell(row, col).text_frame.paragraphs[0].font.size = Pt(8)
            summarytable.cell(row,col).text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
        elif row in [3,10,11,12,19,20,21]:
            summarytable.cell(row, col).text_frame.text = str(round((summary_table_clean.iloc[row-1, col]),1))+'%'
            summarytable.cell(row, col).text_frame.paragraphs[0].font.size = Pt(8)
            summarytable.cell(row,col).text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
        elif row in[0]:
            replace_text('10a', summary_scenario_one_shift)
            replace_text('10b', summary_scenario_two_shift)
            replace_text('10c', summary_scenario_three_shift)
            summarytable.cell(row, col).text_frame.paragraphs[0].font.size = Pt(8)
            summarytable.cell(row,col).text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
            




In [56]:
create_footnote(summaryslide)

In [57]:
data_source

'provided by Google'

# Search and Replace

In [58]:
replace_text('FUSION PERIOD', fusion_period)
replace_text('DATE',current_month)
replace_text('MM/DD/YYYY - MM/DD/YYYY', campaign_dates)
replace_text('FC',frequency_cap)
replace_text('Campaign XYZ',campaign_name)
replace_text('Target Demo',target_demo)
replace_text('target demo',target_demo)
replace_text('TARGET DEMO',target_demo.upper())
replace_text('2a', original_total_budget)
replace_text('2b', scenarioone_total_budget)
replace_text('2d', scenariotwo_total_budget)
replace_text('2f', scenariothree_total_budget)
replace_text('2c', scenarioone_yt_budget)
replace_text('2e', scenariotwo_yt_budget)
replace_text('2g', scenariothree_yt_budget)
replace_text('ADVERTISER', advertiser)
replace_text('3a',exec_summary_callout_one)
replace_text('3b', exec_summary_callout_two)
replace_text('3c',exec_summary_callout_three)
replace_text('4a',full_reach_callout)
replace_text('5a',summary_scenario_max_shift)
replace_text('5b', reach_data_callout)
replace_text('5c', scenarios_total)
replace_text('5d', original_tv)
replace_text('5e', original_yt)
replace_text('5f', scenario_one_tv)
replace_text('5g', scenario_one_yt)
replace_text('5h', scenario_two_tv)
replace_text('5i', scenario_two_yt)
replace_text('5j', scenario_three_tv)
replace_text('5k', scenario_three_yt)
replace_text('6a', frequency_callout_increaseordecrease)
replace_text('6b', frequency_callout)
replace_text('7a', cost_per_reach_callout)
replace_text('TVDATASOURCE',data_source)
replace_text('PREPAREDFOR',prepared_for)



In [59]:
prs.save(f"/Users/resissma2001/Desktop/Google Share Shift-{campaign_name}.pptx")